In [ ]:
import yaml
import numpy as np
from components.worker import Worker
from components.user import User
from components.outside_expert import OutsideExpert

# Load config file into dictionary
with open("./config/config.yaml", "r") as file:
    config = yaml.safe_load(file)

# Load worker prompts
with open("./config/worker_prompt_examples.yaml", "r") as file:
    worker_prompt_examples = yaml.safe_load(file)

# Initialise base Worker as baseline
base_worker = Worker(config["model_id"], bnb_params=config["bnb_params"])
base_worker.load_model()
base_worker.create_prompt_template(worker_prompt_examples)

# Initialise adapter Worker
adapted_worker = Worker(config["model_id"], lora_id=config["lora_id"], bnb_params=config["bnb_params"])
adapted_worker.load_model()
adapted_worker.create_prompt_template(worker_prompt_examples)

# Initialise adapter Worker 2
adapted_worker_2 = Worker(config["model_id"], lora_id=config["lora_test_id"], bnb_params=config["bnb_params"])
adapted_worker_2.load_model()
adapted_worker_2.create_prompt_template(worker_prompt_examples)

# Initialise User
user = User(data_path=config["user_data_path"])

# Initialise OutsideExpert
with open(config["openai_key_path"], "r") as file:
    openai_key = file.read().strip()
with open("./config/expert_prompt_examples.yaml", "r") as file:
    expert_prompt_examples = yaml.safe_load(file)

helpfulness_expert = OutsideExpert(
    model_id=config["expert_model_id"],
    openai_key=openai_key,
    target="helpfulness",
    example=expert_prompt_examples["helpfulness"],
)

In [ ]:
base_ratings = []
adapted_ratings = []
adapted_ratings_2 = []

for step in range(config["num_steps"]):

    print(f"Step {step+1}/{config['num_steps']}:")

    # Get user query
    query = user.get_query()
    print(query)

    # Get base worker response
    base_response = base_worker.generate_text([query])[0]
    print(base_response)
    
    # Get helpfulness rating
    base_rating = helpfulness_expert.rate_responses(
        [query], [base_response]
    )[0]
    base_ratings.append(base_rating)

    # Get adapted worker response
    adapted_response = adapted_worker.generate_text([query])[0]
    print(adapted_response)

    # Get helpfulness rating
    adapted_rating = helpfulness_expert.rate_responses(
        [query], [adapted_response]
    )[0]
    adapted_ratings.append(adapted_rating)

    # Get adapted worker 2 response
    adapted_response_2 = adapted_worker_2.generate_text([query])[0]
    print(adapted_response_2)

    # Get helpfulness rating
    adapted_rating_2 = helpfulness_expert.rate_responses(
        [query], [adapted_response_2]
    )[0]
    adapted_ratings_2.append(adapted_rating_2)

print(f"Base helpfulness rating mean = {np.mean(base_ratings)}")
print(f"Adapted helpfulness rating mean = {np.mean(adapted_ratings)}")
print(f"Adapted 2 helpfulness rating mean = {np.mean(adapted_ratings_2)}")

In [ ]:
import matplotlib.pyplot as plt

# Data for the line chart
x = [0, 225*8*4, 225*32*4]
y = [np.mean(base_ratings), np.mean(adapted_ratings), np.mean(adapted_ratings_2)]

# Create the line chart
plt.plot(x, y, marker='o')

# Adding labels and title
plt.xlabel('Number samples instruction tuned')
plt.ylabel('Mean helpfulness rating')
plt.title('Increase in helpfulness with instruction tuning')

# Show the plot
plt.show()